##### In questa sezione importiamo tutte le librerie necessarie: `face_recognition` per l'elaborazione dei volti, `cv2` per la gestione dei flussi video tramite OpenCV, e `numpy` per operazioni matematiche e gestione dei dati numerici.

In [ ]:
import face_recognition
import cv2
import numpy as np

## Inizializziamo la videocamera integrata (indice 0), necessaria per acquisire in tempo reale i frame video da analizzare per il riconoscimento facciale.

In [ ]:
video_capture = cv2.VideoCapture(0)

## Carichiamo le immagini di riferimento e ne estraiamo l'embedding facciale. Questi vettore numerico rappresenta le caratteristiche distintive di ogni volto.

In [ ]:
stefano_image = face_recognition.load_image_file("dataset/stenni.jpg")
stefano_face_encoding = face_recognition.face_encodings(stefano_image)[0]

## Load a second sample picture and learn how to recognize it.

In [ ]:
raoul_image = face_recognition.load_image_file("dataset/raoul.jpg")
raoul_face_encoding = face_recognition.face_encodings(raoul_image)[0]

## Carica una seconda immagine e impara a riconoscerla.

In [ ]:
antonio_image = face_recognition.load_image_file("dataset/antonio.jpg")
antonio_face_encoding = face_recognition.face_encodings(antonio_image)[0]

## Creiamo due array: uno contenente gli embedding dei volti noti e l'altro i relativi nomi. Questi serviranno per confrontare e riconoscere i volti nei frame successivi.

In [ ]:
known_face_encodings = [
    stefano_face_encoding,
    raoul_face_encoding,
    antonio_face_encoding
]
known_face_names = [
    "Stefano",
    "Raoul",
    "Antonio"
]

## Inizializziamo le strutture dati che memorizzeranno, per ogni frame, le posizioni dei volti, i relativi embedding e i nomi riconosciuti.

In [ ]:
face_locations = []
face_encodings = []
face_names = []

### Loop principale del riconoscimento in tempo reale:
- Acquisizione del frame dalla webcam;
- Estrazione e codifica dei volti;
- Confronto con i volti noti per determinarne l'identità;
- Visualizzazione dei risultati con riquadri e nomi associati ai volti.

In [ ]:
while True:
    # Leggi un singolo frame dalla webcam
    ret, frame = video_capture.read()
    if not ret:
        break

    # 0 per flipping verticale
    # frame = cv2.flip(frame, 0)

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb)

    if face_locations:
        face_encodings = face_recognition.face_encodings(rgb, face_locations)

        face_names = []

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Sconosciuto"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            confidence = 1 - face_distances[best_match_index]  # valore tra 0 e 1

            if matches[best_match_index] and confidence >= 0.1:
                name = f"{known_face_names[best_match_index]} ({int(confidence * 100)}%)"

            face_names.append((name, (top, right, bottom, left)))  # salva anche le coordinate

    # Display the results
    for name, (top, right, bottom, left) in face_names:
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 255, 255), 1)
        # cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Mostra il frame risultante
    cv2.imshow('Video - Premi Q per uscire', frame)

    # Esci se premi 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Rilascia la webcam e chiudi le finestre
video_capture.release()
cv2.destroyAllWindows()